#### 1. Configurando o ambiente

In [1]:
# Instalando os pacotes necessários

# pip install -U ipykernel    # Kernel do Jupyter
# pip install -U jupyter      # Jupyter
# pip install openai          # API do OpenAI
# pip install flask-cors      # CORS para o Flask

In [ ]:
# Configuração básica

import os                                               # Biblioteca para manipulação de arquivos
from openai import OpenAI                               # Biblioteca para uso da API do OpenAI       
import json                                             # Biblioteca para manipulação de arquivos JSON

# Carregando variáveis de ambiente

with open('azure_openai_keys.json', 'r') as f:          # Carregando as chaves de API   
    keys = json.load(f)                                                                         

# Configuração da API 

api_key = keys["AZURE_OPENAI_API_KEY"]                  # Chave de API do OpenAI
api_base = keys["AZURE_OPENAI_ENDPOINT"]                # Endpoint da API  (azure, gcp, aws)

#### 2. Chamada da API

In [6]:
# Exemplo de chamada à API para geração de texto

# Inicialização do cliente OpenAI com as configurações

client = OpenAI(
    api_key=api_key,                            # Chave de API
    base_url=api_base,                          # Endpoint da API
)

# Chamada à API para geração de texto

try:
    response = client.completions.create(
        model="gpt-4",                         # Nome do modelo
        temperature=0.7,                        # Controla a criatividade do texto gerado  (0.0 - 1.0)
        max_tokens=100,                         # Número máximo de tokens a serem gerados no texto
        prompt="Explique o que é Inteligência Artificial Generativa."      # Texto de entrada para geração de texto
    )

    print(response.choices[0].text.strip())     # Exibindo o texto gerado pela API

except Exception as e:
    print(f"Erro ao chamar a API: {e}")         # Exibindo o erro caso ocorra   

Erro ao chamar a API: Error code: 404 - {'error': {'code': '404', 'message': 'Resource not found'}}


#### 3. Exibir modelos disponiveis 

In [ ]:
# Listando modelos disponíveis

try:
    models = client.models.list()                       # Chamada à API para listar os modelos disponíveis
    for model in models:                                # Iterando sobre os modelos
        print(model.id)
        
except Exception as e:
    print(f"Erro ao listar modelos: {e}")               # Exibindo o erro caso ocorra

Erro ao listar modelos: name 'client' is not defined


#### 4. Personalizar respostas com parâmetros

In [2]:
# Ajustando parâmetros para personalizar a resposta

try:
    response = client.completions.create(
        model="gpt-4",                                              # Atualizado nome do modelo
        prompt="Escreva uma história curta sobre uma Capivara.",    # Texto de entrada para geração de texto
        temperature=0.5,                                            # Controla a criatividade do texto gerado  (0.0 - 1.0)       
        max_tokens=150,                                             # Número máximo de tokens a serem gerados no texto
        top_p=1.0,                                                  # Controla a probabilidade de escolha de tokens
        frequency_penalty=0.5,                                      # Controla a frequência de palavras repetidas
        presence_penalty=0.5                                        # Controla a presença de palavras repetidas
    )

    print(response.choices[0].text.strip())                         # Exibindo o texto gerado pela API

except Exception as e:                                              # Exibindo o erro caso ocorra
    print(f"Erro na geração de texto: {e}")                         

Erro na geração de texto: name 'client' is not defined


#### 5. Trabalhando com conversas (chatbots)

In [6]:
# Exemplo de conversa com o modelo

messages = [    
    {"role": "system", "content": "Você é um especialista em pescarias."},
    {"role": "user", "content": "De onde é o peixe Tambaqui?"}
]

try:
    response = client.chat.completions.create(
        model="gpt-4",             # Nome do modelo
        messages=messages,          # Mensagens trocadas entre o usuário e o modelo
        temperature=0.7,           # Controla a criatividade do texto gerado
        max_tokens=100            # Número máximo de tokens a serem gerados
    )

    print(response.choices[0].message.content)   # Exibindo a resposta do modelo

except Exception as e:
    print(f"Erro na conversação: {e}")

Erro na conversação: Error code: 404 - {'error': {'code': '404', 'message': 'Resource not found'}}


#### 6. Integração com aplicações

In [ ]:
from flask import Flask, request, jsonify
from openai import OpenAI
from flask_cors import CORS

# Inicializando a aplicação Flask
app = Flask(__name__)
CORS(app)

# Inicializando o cliente OpenAI com verificação de credenciais
try:
    client = OpenAI(
        api_key=keys["AZURE_OPENAI_API_KEY"],
        base_url=keys["AZURE_OPENAI_ENDPOINT"]
    )
except KeyError as e:
    print(f"Erro: Credencial não encontrada - {e}")
    raise

@app.route('/generate', methods=['POST'])
def generate_text():
    try:
        data = request.json
        if not data or 'prompt' not in data:
            return jsonify({"error": "Prompt não fornecido"}), 400
        
        # Usando chat completion ao invés de completion
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "user", "content": data.get('prompt')}
            ],
            temperature=0.7,
            max_tokens=100
        )

        return jsonify({
            "success": True,
            "response": response.choices[0].message.content
        })

    except Exception as e:
        return jsonify({
            "success": False,
            "error": str(e)
        }), 500

if __name__ == '__main__':
    app.run(debug=True, port=5000)

SyntaxError: invalid syntax (2996928757.py, line 3)